In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def read_data(filename):
    df = pd.read_csv(filename, encoding='utf-8', sep=',')
    return df


def calc_cation(df):
    molsi = df['SiO2'] / 60.0843
    molti = df['TiO2'] / 79.8788
    molal = df['Al2O3'] / 101.9613
    molfe = df['FeO'] / 71.8464
    molmn = df['MnO'] / 70.93745
    molmg = df['MgO'] / 40.304
    molca = df['CaO'] / 56.077
    molna = df['Na2O'] / 61.9789
    molk  = df['K2O'] / 94.195
    molcr = df['Cr2O3'] / 151.9904
    molni = df['NiO'] / 74.6894
    coef = 6 / ( 2 * molsi + 2 * molti + 3 * molal + molfe + molmn + molmg + molca + molna + molk + 3 * molcr + molni )
    
    si = molsi * coef
    ti = molti * coef
    al = 2 * molal * coef
    fe = molfe * coef
    mn = molmn * coef
    mg = molmg * coef
    ca = molca * coef
    na = 2 * molna * coef
    k  = 2 * molk  * coef
    cr = 2 * molcr * coef
    ni = molni * coef
    cation = si + ti + al + fe + mn + mg + ca + na + k + cr + ni    
    return cation, si, ti, al, fe, mn, mg, ca, na, k, cr, ni


def alconvert(si, al):
    al_iv = 2 - si
    al_vi = al - al_iv
    return al_iv, al_vi


def cpx(si, al, fe, mn, mg, ca, na, cr):
    al_iv, al_vi = alconvert(si, al)
    cats = al_vi - na
    cati = 0.5 * ( al_iv - cats )
    crcats = 0.5 * cr
    dihd = ca - cati - cats - crcats
    enfs_cpx = 0.5 * ( fe + mg - dihd )
    mgnum_cpx = mg / ( mg + fe )
    di = dihd * ( mg / (mg + mn + fe ) )
    return enfs_cpx, mgnum_cpx, di


def opx(si, ti, al, fe, mn, mg, ca, na, cr):
    al_iv, al_vi = alconvert(si, al)
    fmal2 = al_vi - na - cr
    enfs_opx = 0.5 * ( mg + mn + fe - ( ti + fmal2 + ca ) )
    return enfs_opx, fmal2


def calctemp(enfs_cpx, enfs_opx, mg_cpx, mgnum_cpx, mn_opx, fmal2, di, pressure):
    putirka = 13.4 - 3.4*np.log(enfs_cpx/enfs_opx) + 5.59*np.log(mg_cpx) - 8.8*mgnum_cpx \
    + 23.85*mn_opx + 6.48*fmal2 - 2.38*di - 0.044*pressure
    tempc = 10000 / putirka
    print(tempc)
    return tempc


def main():
    pressure_mpa = 200
    pressure = 0.01 * pressure_mpa
    
    file_cpx, file_opx = 'cpx.csv', 'opx.csv'
    
    # read csv data
    df_cpx = read_data(file_cpx)
    cation_cpx, si_cpx, ti_cpx, al_cpx, fe_cpx, mn_cpx, mg_cpx, ca_cpx, na_cpx, k_cpx, cr_cpx, ni_cpx = calc_cation(df_cpx)
    df_opx = read_data(file_opx)
    cation_opx, si_opx, ti_opx, al_opx, fe_opx, mn_opx, mg_opx, ca_opx, na_opx, k_opx, cr_opx, ni_opx = calc_cation(df_opx)
    # calc parameters
    enfs_cpx, mgnum_cpx, di = cpx(si_cpx, al_cpx, fe_cpx, mn_cpx, mg_cpx, ca_cpx, na_cpx, cr_cpx)
    enfs_opx, fmal2 = opx(si_opx, ti_opx, al_opx, fe_opx, mn_opx, mg_opx, ca_opx, na_opx, cr_opx)
    # estimate temperature
    temp = calctemp(enfs_cpx, enfs_opx, mg_cpx, mgnum_cpx, mn_opx, fmal2, di, pressure)
    temp.to_csv('result.csv')
    
if __name__ == "__main__":
    main()

0      963.544313
1     1205.872886
2     1210.678891
3      973.416066
4      984.202477
5     1250.291545
6     1206.636106
7     1272.062335
8     1265.147661
9     1271.378170
10    1282.148533
11    1254.237542
12    1118.708364
13    1174.564361
14    1236.359811
15    1239.306946
16    1260.668736
17    1217.860307
18    1243.415365
19    1254.997548
20    1262.386600
21    1315.752088
22    1183.222751
23    1209.392903
24     802.112886
dtype: float64
